In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import re
from googlesearch import search
#Aug 2019 to Oct 2021 (26 month period)
#twitch = twitch.drop(['UserID'], axis=1)

In [2]:
twitch = pd.read_csv(r'C:\Users\harac\OneDrive\Documents\DAS\twitchpay.csv')

In [3]:
twitch

,Rank,Username,UserID,GrossEarning
0,1,CriticalRole,229729353.0,9626712.16
1,2,xQcOW,71092938.0,8454427.17
2,3,summit1g,26490481.0,5847541.17
3,4,Tfue,60056333.0,5295582.44
4,5,NICKMERCS,15564828.0,5096642.12
...,...,...,...,...
9995,9996,Quig,114849984.0,23528.80
9996,9997,BoozyCruzy,142789400.0,23527.84
9997,9998,Cewpins,216590269.0,23526.21
9998,9999,Megasnoop,66086072.0,23520.87


In [4]:
twitch.shape
#dimensions of the dataset

(10000, 4)

In [5]:
twitch.count()

Rank            10000
Username        10000
UserID            108
GrossEarning    10000
dtype: int64

In [6]:
twitch.isna().sum()
# a lot of the userID's are missing in the csv, not needed anyway

Rank               0
Username           0
UserID          9892
GrossEarning       0
dtype: int64

In [7]:
twitch.drop(['UserID'], inplace=True, axis=1)

In [8]:
twitch.dtypes

Rank              int64
Username         object
GrossEarning    float64
dtype: object

In [9]:
twitch = twitch.set_index('Rank')

In [10]:
twitch['Username'].loc[twitch['Username'] == "__unknown__"]
#85 values where the twitch user was not present in the twitch csv, 

Rank
11      __unknown__
623     __unknown__
725     __unknown__
809     __unknown__
856     __unknown__
           ...     
9229    __unknown__
9322    __unknown__
9689    __unknown__
9803    __unknown__
9809    __unknown__
Name: Username, Length: 85, dtype: object

In [11]:
twitch = twitch.query("Username != '__unknown__'")

In [12]:
twitch

,Username,GrossEarning
Rank,,
1,CriticalRole,9626712.16
2,xQcOW,8454427.17
3,summit1g,5847541.17
4,Tfue,5295582.44
5,NICKMERCS,5096642.12
...,...,...
9996,Quig,23528.80
9997,BoozyCruzy,23527.84
9998,Cewpins,23526.21


In [13]:
Rank = np.arange(start=1, stop=9916, step=1)

In [14]:
twitch['Rank'] = Rank.tolist()

In [15]:
twitch = twitch.set_index('Rank')

In [16]:
twitch

,Username,GrossEarning
Rank,,
1,CriticalRole,9626712.16
2,xQcOW,8454427.17
3,summit1g,5847541.17
4,Tfue,5295582.44
5,NICKMERCS,5096642.12
...,...,...
9911,Quig,23528.80
9912,BoozyCruzy,23527.84
9913,Cewpins,23526.21


In [17]:
twitchS = pd.Series(twitch['Username'])
#creating a series of the usernames to create HTTP requests to scrape more data on the users

In [18]:
twitchS

Rank
1       CriticalRole
2              xQcOW
3           summit1g
4               Tfue
5          NICKMERCS
            ...     
9911            Quig
9912      BoozyCruzy
9913         Cewpins
9914       Megasnoop
9915     Dwarf_Mamba
Name: Username, Length: 9915, dtype: object

In [19]:
twitchS[twitchS.str.contains("^(?!.*^[a-zA-Z0-9_]*$).*$")]
#Negative lookahead for any rows that do not contain alphanumeric numbers

Rank
74      Riot Games
175           らっだぁ
357          餐餐自由配
521             懶貓
523            풍월량
           ...    
9714        ちゃーちゃん
9738         はるきくん
9801           배린희
9867           딩셉션
9872            冠賢
Name: Username, Length: 241, dtype: object

In [20]:
FixUrls = pd.Series(twitchS[twitchS.str.contains("^(?!.*^[a-zA-Z0-9_]*$).*$")])

In [21]:
placeholder = '{FixUrls} twitchtracker games'

In [22]:
SearchUrl = []
for x in FixUrls:
    fixed = placeholder.format(FixUrls = x)
    SearchUrl.append(fixed)

In [23]:
SearchUrl

['Riot Games twitchtracker games',
 'らっだぁ twitchtracker games',
 '餐餐自由配 twitchtracker games',
 '懶貓 twitchtracker games',
 '풍월량 twitchtracker games',
 '빢유환 twitchtracker games',
 '서새봄냥 twitchtracker games',
 '악어 twitchtracker games',
 '견자희 twitchtracker games',
 'ぐちつぼ twitchtracker games',
 '양띵 twitchtracker games',
 '依渟 twitchtracker games',
 '한동숙 twitchtracker games',
 '嬌兔 twitchtracker games',
 '탬탬버린 twitchtracker games',
 '강지 twitchtracker games',
 '老皮 twitchtracker games',
 'FACEIT TV twitchtracker games',
 'スタンミ twitchtracker games',
 '김진우 twitchtracker games',
 '침착맨 twitchtracker games',
 '꽃핀 twitchtracker games',
 '우왁굳 twitchtracker games',
 '接接 twitchtracker games',
 '蝦愛橘子 twitchtracker games',
 '오킹 twitchtracker games',
 '김도 twitchtracker games',
 '괴물쥐123 twitchtracker games',
 '고차비 twitchtracker games',
 '彩學 twitchtracker games',
 '빛베리 twitchtracker games',
 '따효니 twitchtracker games',
 '나나양 twitchtracker games',
 'さくらみこ twitchtracker games',
 '殺梗 twitchtracker games',
 '狂暴小建 

In [24]:
find = 'らっだぁ twitchtracker games'

In [25]:
cleanedtt = []

In [26]:
def Searchweb():
    for f in search(find, tld="co.in", num=1, stop=1, pause=3):
        print(f)
    if f.endswith("/games"):
        cleanedtt.append(f)
        return
    elif f.endswith("clips"):
        f.replace("clips", "games")
    else: fc = f + "/games"
    cleanedtt.append(fc)
    return
#Asian streamers names are in english, this function searches for their twitch 

In [27]:
Searchweb()

https://twitchtracker.com/raderaderader/games


In [28]:
cleanedtt

['https://twitchtracker.com/raderaderader/games']

In [29]:
ac = pd.read_csv(r'C:\Users\harac\OneDrive\Documents\DAS\Asian characters.csv')

In [30]:
ac = ac.set_index('Rank')

In [31]:
ac = ac.squeeze('columns')

In [32]:
placeholder = 'https://twitchtracker.com/{twitchS}/games'

In [33]:
URLstore = []
for x in twitchS:
    stored = placeholder.format(twitchS = x)
    URLstore.append(stored)

In [34]:
URLstore

['https://twitchtracker.com/CriticalRole/games',
 'https://twitchtracker.com/xQcOW/games',
 'https://twitchtracker.com/summit1g/games',
 'https://twitchtracker.com/Tfue/games',
 'https://twitchtracker.com/NICKMERCS/games',
 'https://twitchtracker.com/ludwig/games',
 'https://twitchtracker.com/TimTheTatman/games',
 'https://twitchtracker.com/Altoar/games',
 'https://twitchtracker.com/auronplay/games',
 'https://twitchtracker.com/LIRIK/games',
 'https://twitchtracker.com/Gaules/games',
 'https://twitchtracker.com/HasanAbi/games',
 'https://twitchtracker.com/Asmongold/games',
 'https://twitchtracker.com/loltyler1/games',
 'https://twitchtracker.com/RanbooLive/games',
 'https://twitchtracker.com/MontanaBlack88/games',
 'https://twitchtracker.com/ibai/games',
 'https://twitchtracker.com/Castro_1021/games',
 'https://twitchtracker.com/MOONMOON/games',
 'https://twitchtracker.com/TheRealKnossi/games',
 'https://twitchtracker.com/moistcr1tikal/games',
 'https://twitchtracker.com/Mizkif/games',

In [35]:
URLstore = pd.Series(URLstore)

In [36]:
URLstore.update(pd.Series(ac))

In [37]:
URLstore
# all asian streamers have URLs in english

0       https://twitchtracker.com/CriticalRole/games
1              https://twitchtracker.com/xQcOW/games
2           https://twitchtracker.com/summit1g/games
3               https://twitchtracker.com/Tfue/games
4          https://twitchtracker.com/NICKMERCS/games
                            ...                     
9910            https://twitchtracker.com/Quig/games
9911      https://twitchtracker.com/BoozyCruzy/games
9912         https://twitchtracker.com/Cewpins/games
9913       https://twitchtracker.com/Megasnoop/games
9914     https://twitchtracker.com/Dwarf_Mamba/games
Length: 9915, dtype: object

In [38]:
mostplayed = []

In [39]:
ac
#8045
#8683 `

Rank
73             https://twitchtracker.com/riotgames/games
174        https://twitchtracker.com/raderaderader/games
356     https://twitchtracker.com/blusewilly_retry/games
520            https://twitchtracker.com/failverde/games
522         https://twitchtracker.com/hanryang1125/games
                              ...                       
9713           https://twitchtracker.com/chatochan/games
9737            https://twitchtracker.com/hal__ism/games
9800          https://twitchtracker.com/heehee1004/games
9866         https://twitchtracker.com/dingception/games
9871             https://twitchtracker.com/gunguno/games
Name: Links, Length: 241, dtype: object

In [40]:
URLstore.index += 1
#changes the index to start at 1

In [41]:
URLstore[1559]

'https://twitchtracker.com/Lil_Lexi/games'

In [42]:
#URLstore.drop(labels=[8046, 8664], inplace=True)

In [43]:
URLstore

1       https://twitchtracker.com/CriticalRole/games
2              https://twitchtracker.com/xQcOW/games
3           https://twitchtracker.com/summit1g/games
4               https://twitchtracker.com/Tfue/games
5          https://twitchtracker.com/NICKMERCS/games
                            ...                     
9911            https://twitchtracker.com/Quig/games
9912      https://twitchtracker.com/BoozyCruzy/games
9913         https://twitchtracker.com/Cewpins/games
9914       https://twitchtracker.com/Megasnoop/games
9915     https://twitchtracker.com/Dwarf_Mamba/games
Length: 9915, dtype: object

In [44]:
URLstore[2]

'https://twitchtracker.com/xQcOW/games'

In [45]:
URLstore50 = URLstore[1]

In [46]:
def LoopURL(URLstore50):
    for x in URLstore50:     
        headers = {"User-Agent": "placeholder"}
        page = requests.get(x, headers=headers)
        soup = BeautifulSoup(page.content, "html.parser")
        soup1 = BeautifulSoup(soup.prettify(), "html.parser")
        table = soup1.find(id='games').select('a', limit=1)
        value = table[0].get_text()
        trueValue = str(value)
        trueValue = re.sub('\s\s+','', trueValue)
        mostplayed.append(trueValue)
    return mostplayed

In [47]:
LoopURL(URLstore50)

MissingSchema: Invalid URL 'h': No schema supplied. Perhaps you meant http://h?

In [49]:
mostplayed

[]